In [1]:
# !pip install opencv-python
# !pip install nltk
# !pip install imutils

In [1]:
import argparse
from collections import defaultdict, Counter
import cv2
from fuzzywuzzy import fuzz
from itertools import combinations # not mandatory
import imutils
from imutils import build_montages, paths
import os
import matplotlib.pyplot as plt
import networkx as nx
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import re
import seaborn as sns

from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder
import string

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import xgboost
from xgboost import cv, XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import os
import shutil
from sklearn.model_selection import cross_val_score

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from imblearn.pipeline import Pipeline as ImbPipeline

2023-08-24 22:38:06.131703: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Define useful functions for cleaning the data

In [2]:
def remove_parentheses(string):
    pattern = r'\([^()]*\)'  # Matches "(...)" pattern
    while re.search(pattern, string):
        string = re.sub(pattern, '', string)
    return string.strip()

def format_ingredients(string):
    string = string.replace('.', '')  # Remove periods
    string = re.sub(r'\s*,\s*', ',', string)  # Remove spaces after commas
    ingredients_list = string.split(',')  # Split by comma
    formatted_ingredients = [ingredient.strip() for ingredient in ingredients_list]  # Remove leading/trailing spaces for each ingredient
    return ', '.join(formatted_ingredients)  # Join formatted ingredients with commas

def clean_text_values(df):
    text_columns = df.select_dtypes(exclude=[np.number]).columns
    for column in text_columns:
        if column == 'category':
            continue
        df[column].fillna('NA', inplace=True)
        df[column] = df[column].map(str.lower)
        if column == 'ingredients':
            df[column] = df[column].apply(remove_parentheses)
            df[column] = df[column].apply(format_ingredients)
        if column == 'household_serving_fulltext':
            df[column] = df[column].map(lambda x: re.sub('[^a-z]+', '', x))
    return df

### Read the data

In [3]:
food_train = pd.read_csv('data/food_train.csv')
food_test = pd.read_csv('data/food_test.csv')
food_nutrients = pd.read_csv('data/food_nutrients.csv')
nutrients_names = pd.read_csv('data/nutrients.csv')

In [4]:
clean_text_values(food_train)
clean_text_values(food_test)
nutrients = pd.merge(food_nutrients, nutrients_names, how='left',on='nutrient_id')
pivoted_nutrients = pd.pivot_table(nutrients, values='amount', index='idx', columns='name')
data = pd.merge(food_train, pivoted_nutrients, how='left', on='idx')
data_test = pd.merge(food_test, pivoted_nutrients, how='left', on='idx')

In [5]:
data_test.head()

,idx,brand,description,ingredients,serving_size,serving_size_unit,household_serving_fulltext,"Alcohol, ethyl",Biotin,"Calcium, Ca",...,"Vitamin A, IU",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid","Vitamin D (D2 + D3), International Units",Vitamin E,Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),Xylitol,"Zinc, Zn"
0,13,"csm bakery products na, inc.",powdered sugar brownie bites,"sugar, enriched bleached flour, water, palm oi...",39.0,g,grm,NaN,NaN,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,15,integrated global organics company llc,baked organic potato snack,"potato starch*, potato flakes*, corn flour*, s...",28.0,g,onz,NaN,NaN,143.0,...,0.0,NaN,NaN,4.3,NaN,NaN,NaN,NaN,NaN,NaN
2,16,jovial,organic einkorn cookies,"organic einkorn flour, organic cane sugar, * o...",28.0,g,cookies,NaN,NaN,36.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,23,"safeway, inc.",white corn bite size tortilla chips,"white corn, sunflower oil and/or corn oil, salt",28.0,g,chips,NaN,NaN,71.0,...,0.0,NaN,NaN,4.3,NaN,NaN,NaN,NaN,NaN,NaN
4,40,"inventure foods, inc.",potato chips,"potatoes, sunflower and/or safflower oil, malt...",28.0,g,onz,NaN,NaN,0.0,...,0.0,NaN,NaN,21.4,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
cols_to_remove = data.columns[data.isnull().mean() > 0.8]
data = data.drop(columns=cols_to_remove)

cols_to_remove_test = data_test.columns[data_test.isnull().mean() > 0.8]
data_test = data_test.drop(columns=cols_to_remove_test)

### ENUMS

In [7]:
CANDY = 'candy'
COOKIES = 'cookies_biscuits'
CAKES = 'cakes_cupcakes_snack_cakes'
CHIPS_PRETZELS = 'chips_pretzels_snacks'
CHOCOLATE = 'chocolate'
POPCORN_PEANUTS = 'popcorn_peanuts_seeds_related_snacks'

## Let's start with the photos
### We will build a CNN, then calculating the probabilities vector and adding it to the tabular data

In [ ]:
# Set the paths and image size
train_data_dir = 'images/train'
image_dir = 'images/train'
image_size = (128, 128)

# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split data into training and validation
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)


# Load and preprocess training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Use the training subset of data
)


# Load and preprocess validation data
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use the validation subset of data
)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(6, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
num_epochs = 10
model.fit(train_generator, epochs=num_epochs, validation_data=validation_generator)

# Evaluate the model on test data
test_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use the validation subset of data for testing
)

eval_result = model.evaluate(test_generator)
print("Test Loss:", eval_result[0])
print("Test Accuracy:", eval_result[1])


# Save the model
model.save('cnn_model.h5')

## We wrote a regular .py script that will export the probabilitis vector to .csv file
## Lets read it and add it to the tabular data

In [8]:
photos_probs = pd.read_csv('photos_probs.csv')
idx = photos_probs['idx'].unique()
print(len(idx))

22077


### We didnt get probs for all the 30K rows,
### So we have 2 df, one with 30K rows and without the images probs
### The second will have 20K rows but with images probs

### We will decide based on the following results on which we should foucus

In [9]:
data_with_probs = pd.merge(data, photos_probs, how="left",on='idx')

In [12]:
df1 = data.copy()
df2 = data_with_probs[data_with_probs['idx'].isin(idx)].copy()

In [13]:
df1['category'].value_counts()

popcorn_peanuts_seeds_related_snacks    7645
candy                                   7584
cookies_biscuits                        5284
cakes_cupcakes_snack_cakes              3786
chocolate                               3772
chips_pretzels_snacks                   3680
Name: category, dtype: int64

### We'll start with splitting df1 into train_test
### Also we'll split the train set into 2 for feature eng' and model tuning
### We will use the fe also for fillNA methods, and the modeltuning set for resampling

In [14]:
X = df1.loc[:,df1.columns != 'category']
y = df1.loc[:,['category']]

y['category'].replace(['cakes_cupcakes_snack_cakes', 'candy', 'chips_pretzels_snacks', 'chocolate',
 'cookies_biscuits', 'popcorn_peanuts_seeds_related_snacks'],
                        [0, 1, 2, 3, 4, 5], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_fe, X_mt, y_fe, y_mt = train_test_split(X_train, y_train, test_size=0.4, random_state=42, stratify=y_train)

X_fe_train, X_fe_test, y_fe_train, y_fe_test = train_test_split(X_fe, y_fe, test_size=0.2, random_state=42, stratify=y_fe)
X_mt_train, X_mt_test, y_mt_train, y_mt_test = train_test_split(X_mt, y_mt, test_size=0.2, random_state=42, stratify=y_mt)

In [15]:
X_y_train = pd.merge(X_train, y_train, how="left",left_index=True, right_index=True)
X_y_train['idx'] = X_y_train['idx'].map(lambda x: str(x))
X_y_test = pd.merge(X_test, y_test, how="left",left_index=True, right_index=True)
X_y_test['idx'] = X_y_test['idx'].map(lambda x: str(x))


X_y_mt = pd.merge(X_mt, y_mt, how="left",left_index=True, right_index=True)


#### #### concat for eda
X_fe_w_cat = pd.merge(X_fe_train, y_fe_train, how="left",left_index=True, right_index=True)

### Functions for eda

In [16]:
def data_by_category(df, category):
    return df[df['category'] == category]

def better_name(df, col_name):
    df[col_name] = df[col_name].map(lambda x: str(x).translate(str.maketrans('', '', string.punctuation)))
    return(df)

In [17]:
def get_most_popular_words(column):
    text = ', '.join(column.values)
    tokens = [word.strip() for word in text.split(',')]
    stop_words = set(stopwords.words('english'))
    tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
    word_counts = Counter(tokens)
    return word_counts

In [18]:
def select_top_words(row, dict_words, column):
    desc = row[column]
    for category, words in dict_words.items():
        for word in dict_words[category]:
            if word in desc:
                name = f"{column}_{category}_{word}"
                return name
    return 'None'

## BRAND
### We extracting for every category it's most popular brands
# ADD GRAPHS

In [69]:
dict(X_fe_w_cat[X_fe_w_cat['category'] == 1]['brand'].value_counts().head(10))

{'ferrara candy company': 187,
 'frankford candy llc': 63,
 'not a branded item': 56,
 'walgreens co': 48,
 'sunmark': 47,
 'cvs pharmacy inc': 43,
 'russell stover candies inc': 42,
 'ross acquisition inc': 42,
 'mars chocolate north america llc': 41,
 'just born inc': 38}

In [73]:
def get_brands_dict(z):
    d = dict()
    df = X_fe_w_cat
    for cat in df['category'].unique():
        x = df[df['category'] == cat]['brand'].value_counts().head(z)
        d[cat] = x
    return d

In [76]:
q = get_brands_dict(10)
type(q[4])

pandas.core.series.Series

In [16]:
def top_x_brands_by_category(df, category, x):
    df_by_category = df[df['category'] == category]
    return df_by_category.groupby('brand').size().sort_values(ascending=False).head(x)

def select_only_the_top_brand(a):
    df = X_fe_w_cat
    better_name(df, 'brand')
    top_brand_dict={'candy': top_x_brands_by_category(df, CANDY, a).index.tolist(),
                   'cookies': top_x_brands_by_category(df, COOKIES, a).index.tolist(),
                   'cakes': top_x_brands_by_category(df, CAKES, a).index.tolist(),
                   'chips_pretzels': top_x_brands_by_category(df, CHIPS_PRETZELS, a).index.tolist(),
                   'chocolate': top_x_brands_by_category(df, CHOCOLATE, a).index.tolist(),
                   'popcorn_peanuts': top_x_brands_by_category(df, POPCORN_PEANUTS, a).index.tolist()}
    
    return top_brand_dict

In [17]:
def get_top_brands(df, a):
    top_brand_dict = select_only_the_top_brand(a)
    df['brand'].fillna('unbranded',inplace=True)
    brands_list = []
    for key, value in top_brand_dict.items():
        if 'not a branded item' in value:
            value.remove('not a branded item')
        brands_list.extend(value)  
    df['brand'] = df['brand'].map(lambda x: x if x in brands_list else 'unbranded')
    return df

In [18]:
select_only_the_top_brand(10)

{'candy': [],
 'cookies': [],
 'cakes': [],
 'chips_pretzels': [],
 'chocolate': [],
 'popcorn_peanuts': []}

In [19]:
def extract_brand(df):
    brand_words = {
    "candy": [
        "ferrara candy company",
        "frankford candy llc",
        "sunmark",
        "mars chocolate north america llc",
        "just born inc",
        "ross acquisition inc",
        "russell stover candies inc",
        "maud borup inc",
        "reeses",
        "tops markets llc",
        "supervalu inc",
        "weis markets inc",
        "tootsie roll industries inc",
        "holiday candy corp inc",
        "wm wrigley jr company",
    ],
    "cookies": [
        "nabisco biscuit company",
        "keebler company",
        "lofthouse foods",
        "ahold usa inc",
        "bimbo bakeries usa inc",
        "safeway inc",
        "wegmans food markets inc",
        "lenny  larrys",
        "abimar foods inc",
        "kingston marketing co",
    ],
    "cakes": [
        "mckee foods corporation",
        "hostess brands llc",
        "tasty baking company",
        "bimbo bakeries usa inc",
        "sweet ps bake shop",
        "twobite",
        "schnuck markets inc",
        "fresh  easy",
        "dawn food products inc",
        "dierbergs markets inc",
        "flowers foods inc",
        "labrees bakery inc",
        "rich products corporation",
        "aryzta llc",
        "rocky mountain pies",
    ],
    "chips_pretzels": [
        "utz quality foods inc",
        "the hain celestial group inc",
        "herr foods inc",
        "snyderslance inc",
        "jays foods inc",
        "cape cod potato chips inc",
        "inventure foods inc",
        "old dutch foods inc",
        "better made snack foods inc",
        "small planet foods inc",
        "roundys",
        "pinnacle foods group llc",
    ],
    "chocolate": [
        "lindt  sprungli schweiz ag",
        "russell stover candies inc",
        "mars chocolate north america llc",
        "godiva chocolatier inc",
        "ghirardelli chocolate company",
        "frankford candy llc",
        "moonstruck chocolate co",
        "ross acquisition inc",
        "rm palmer co",
        "hammonds candies since 1920 llc",
        "theo chocolate inc",
        "demets candy company",
        "ghirardelli",
        "green  blacks",
        "fannie may confections inc",
        "nestle usa inc",
        "harmons inc",
        "whitmans candies inc",
    ],
    "popcorn_peanuts": [
        "american importing co inc",
        "nabisco food company",
        "john b sanfilippo  son inc",
        "tops markets llc",
        "star snacks co inc",
        "safeway inc",
        "supervalu inc",
        "ahold usa inc",
        "weis markets inc",
    ],
}


    
    num_rows = len(df)
    # Convert the dictionary values to sets to ensure uniqueness
    data_dict = {key: set(values) for key, values in brand_words.items()}
    # Create a new dictionary with the column names as keys and zeros as values
    columns_dict = {f"{key}_{word}": np.zeros(num_rows) for key, words in brand_words.items() for word in words}
    # Create a DataFrame from the dictionary
    new_df = pd.DataFrame(columns_dict)
    # Concatenate the existing DataFrame with the new DataFrame
    result_df = pd.concat([df.reset_index(drop=True), new_df], axis=1)
    result_df.index = df.index

    for key, words in brand_words.items():
        for word in words:
            result_df[f"{key}_{word}"] = result_df['brand'].apply(lambda x: 1 if isinstance(x, str) and word in x else 0)
            
    result_df['brand'] = result_df.apply(lambda row: select_top_words(row, brand_words, 'brand'), axis=1)
    
    return result_df

## Description

# ADD GRPAHS OF POPULAR WORDS

In [20]:
# def get_top_x_words_desc(b):
#     df = X_fe_w_cat
#     category_words = {}
#     desc = dict()
#     description_all_words = set()
    
#     for category, group in df.groupby('category'):
#         category_words[category] = {'description': Counter()}
#         for _, row in group.iterrows():
#             description_word_counts = get_most_popular_words(pd.Series([row['description']]))
#             category_words[category]['description'] += description_word_counts
            
#     for category, words in category_words.items():
#         description_word_counts = words['description']
#         description_top_words = description_word_counts.most_common(b)
#         description_words, description_frequencies = zip(*description_top_words)
#         description_all_words.update(set(description_words))
#         desc[category] = list(description_words)
        
#     return desc

In [21]:
# def desc_indication_columns(df, b):
#     desc = get_top_x_words_desc(b)
#     df['description'].fillna('undescription',inplace=True)
#     for category, words in desc.items():
#         name = f"{category}_desc"
#         df[name] = df['description'].apply(lambda x: 1 if any(word in x.lower() for word in words) else 0)
#     return df

### After analyzing X_fe we got this dict that give indication words for every category
### We will add a column for every word, 1 for if the word is in the desc, else 0

In [22]:
def extract_description(df):
    desc_words = {CAKES:['tastykake', 'cupcakes', 'cheezecake', 'bakery fresh goodness', 'apple pie', 'pie', 'pecan pie',
                                   'pumpkin pie', 'cake', 'cakes', 'cupcake', 'coffeecake', 'brownie', 'brownies', 'slice', 'sliced', 
                                    'torte', 'pies', 'donut', 'donuts', 'bakery'],
    CANDY:['candy', 'candies', 'gummy', 'gummi', 'gummies', 'twist', 'stick', 'marshmallow', 'marshmallows',
                       'jelly', 'snacks', "sweet's",  "brach's", 'cherry', 'strawberry', 'orange', 'watermelon', 
                       'peppermint', 'grape'],
    CHIPS_PRETZELS:['potato chips', 'tortilla chips', 'kettle cooked potato chips', 'sea salt', 'kettle chips', 
                               "snyder's of hanover", 'sour cream & onion', 'wavy potato chips', "herr's", 'chips', 'chip', 
                               'tortilla', 'crisps', 'crisp', 'potato', 'pretzel', 'pretzels', 'fries', 'corn'],
    CHOCOLATE:['dark chocolate', 'lindt', 'ghirardelli chocolate', 'russell stover', 'godiva', 'truffles', 
                          'dark chocolate bar', 'cocoa', 'praline', 'toffee', 'belgian', 'dark', 'truffle', 'chocolates'],
    COOKIES:['cookie', 'cookies', 'chocolate chip', 'chocolate chip cookies', 'sandwich cookies', 
                       'shortbread cookies', 'frosted sugar cookies', 'sugar cookies''crackers', 'frosted', 'wafer', 
                       'cracker', 'biscuit', 'macaroon', 'waffle'],
    POPCORN_PEANUTS:['popcorn', 'almond', 'almonds', 'trail mix', 'peanuts', 'mixed nuts', 
                                'dry roasted peanuts','cashews', 'kernel', 'shell', 'pecan']}
    
    num_rows = len(df)
    # Convert the dictionary values to sets to ensure uniqueness
    data_dict = {key: set(values) for key, values in desc_words.items()}
    # Create a new dictionary with the column names as keys and zeros as values
    columns_dict = {f"{key}_{word}": np.zeros(num_rows) for key, words in desc_words.items() for word in words}
    # Create a DataFrame from the dictionary
    new_df = pd.DataFrame(columns_dict)
    # Concatenate the existing DataFrame with the new DataFrame
    result_df = pd.concat([df.reset_index(drop=True), new_df], axis=1)
    result_df.index = df.index

    for key, words in desc_words.items():
        for word in words:
            result_df[f"{key}_{word}"] = result_df['description'].apply(lambda x: 1 if isinstance(x, str) and word in x else 0)
            
    result_df['description'] = result_df.apply(lambda row: select_top_words(row, desc_words, 'description'), axis=1)
    
    return result_df

## Ingridents
# ADD GRPAHS OF POPULAR WORDS

In [23]:
# def get_top_x_words_ingre(c):
#     df = X_fe_w_cat
#     category_words = {}
#     ingre = dict()
#     ingredietns_all_words = set()
    
#     for category, group in df.groupby('category'):
#         category_words[category] = {'ingredients': Counter()}
#         for _, row in group.iterrows():
#             ingredients_word_counts = get_most_popular_words(pd.Series([row['ingredients']]))
#             category_words[category]['ingredients'] += ingredients_word_counts
            
#     for category, words in category_words.items():
#         ingredients_word_counts = words['ingredients']
#         ingredients_top_words = ingredients_word_counts.most_common(c)
#         ingredients_words, ingredients_frequencies = zip(*ingredients_top_words)
#         ingredietns_all_words.update(set(ingredients_words))
#         ingre[category] = ingredients_words
        
#     return ingre

In [24]:
# def ingre_indication_columns(df,c):
#     ingre = get_top_x_words_ingre(c)
#     for category, words in ingre.items():
#         name = f"{category}_ingre"
#         df[name] = df['ingredients'].apply(lambda x: 1 if any(word in x.lower() for word in words) else 0)
#     return df

### After analyzing X_fe we got this dict that give indication words for every category
### We will add a column for every word, 1 for if the word is in the desc, else 0

In [25]:
def extract_ingredients(df):
    ingre_words = {CAKES:['leavening', 'eggs'],
                   CANDY:['gelatin', 'carnauba wax', 'red 40', 'blue 1'],
                   CHIPS_PRETZELS:['potatoes', 'onion powder', 'garlic powder', 'maltodextrin', 'yeast extract'],
                   CHOCOLATE:['cocoa butter', 'milk chocolate', 'chocolate', 'milk', 'dark chocolate', 'butter'],
                   COOKIES:['baking soda', 'enriched flour', 'leavening', 'eggs'],
                   POPCORN_PEANUTS:['almonds', 'peanuts', 'cashews', 'popcorn', 'pecans']}
    
    num_rows = len(df)
    # Convert the dictionary values to sets to ensure uniqueness
    data_dict = {key: set(values) for key, values in ingre_words.items()}
    # Create a new dictionary with the column names as keys and zeros as values
    columns_dict = {f"{key}_{word}": np.zeros(num_rows) for key, words in ingre_words.items() for word in words}
    # Create a DataFrame from the dictionary
    new_df = pd.DataFrame(columns_dict)
    # Concatenate the existing DataFrame with the new DataFrame
    result_df = pd.concat([df.reset_index(drop=True), new_df], axis=1)
    result_df.index = df.index

    for key, words in ingre_words.items():
        for word in words:
            result_df[f"{key}_{word}"] = result_df['ingredients'].apply(lambda x: 1 if isinstance(x, str) and word in x else 0)
            
    result_df['ingredients'] = result_df.apply(lambda row: select_top_words(row, ingre_words, 'ingredients'), axis=1)
            
    return result_df

## household

# ADD GRPAHS OF POPULAR WORDS

In [26]:
# def get_top_x_words_household(d):
#     df = X_fe_w_cat
#     category_words = {}
#     house = dict()
#     household_all_words = set()
    
#     for category, group in df.groupby('category'):
#         category_words[category] = {'household_serving_fulltext': Counter()}
#         for _, row in group.iterrows():
#             household_word_counts = get_most_popular_words(pd.Series([row['household_serving_fulltext']]))
#             category_words[category]['household_serving_fulltext'] += household_word_counts
            
#     for category, words in category_words.items():
#         household_word_counts = words['household_serving_fulltext']
#         household_top_words = household_word_counts.most_common(d)
#         household_words, household_frequencies = zip(*household_top_words)
#         household_all_words.update(set(household_words))
#         house[category] = household_words
        
#     return house

In [27]:
# def household_indication_columns(df,d):
#     household = get_top_x_words_household(d)
#     for category, words in household.items():
#         name = f"{category}_household"
#         df[name] = df['household_serving_fulltext'].apply(lambda x: 1 if any(word in x.lower() for word in words) else 0)
#     return df

### After analyzing X_fe we got this dict that give indication words for every category
### We will add a column for every word, 1 for if the word is in the desc, else 0

In [28]:
def extract_household(df):
    household_words = {CAKES:['cake', 'cakes', 'cupcakes', 'cupcake','brownie', 'pie', 'donut', 'muffin', 'tart', 
                              'torte', 'doughnut','slice', 'pastry', 'bun', 'loaf'],
                       CANDY:['candies', 'candy','gummies', 'gummy', 'marshmallow', 'pop', 'twist', 'stick','bear'],
                       CHIPS_PRETZELS:['chip', 'chips', 'fries', 'crisp', 'chipsabout', 'pretzelsabout'],
                       CHOCOLATE:['squares', 'square', 'balls', 'ball', 'truffle', 'praline', 'pralines', 'block', 'tablet', 'bar'],
                       COOKIES:['cookies', 'cookie', 'wafers', 'wafer', 'crackers', 'cracker', 'biscuit', 'macaroon', 'waffle'],
                       POPCORN_PEANUTS:['tbsp', 'kernel', 'popcorn', 'almond', 'shell', 'pecan']}
    
    num_rows = len(df)
    # Convert the dictionary values to sets to ensure uniqueness
    data_dict = {key: set(values) for key, values in household_words.items()}
    # Create a new dictionary with the column names as keys and zeros as values
    columns_dict = {f"{key}_{word}": np.zeros(num_rows) for key, words in household_words.items() for word in words}
    # Create a DataFrame from the dictionary
    new_df = pd.DataFrame(columns_dict)
    # Concatenate the existing DataFrame with the new DataFrame
    result_df = pd.concat([df.reset_index(drop=True), new_df], axis=1)
    result_df.index = df.index

    for key, words in household_words.items():
        for word in words:
            result_df[f"{key}_{word}"] = result_df['household_serving_fulltext'].apply(lambda x: 1 if isinstance(x, str) and word in x else 0)
            
    result_df['household_serving_fulltext'] = result_df.apply(lambda row: select_top_words(row, household_words, 'household_serving_fulltext'), axis=1)
            
    return result_df

In [29]:
def drop_columns(df):
    return df.drop(['serving_size_unit'],axis=1)

# images --- need to delete this at the end

In [ ]:
def path(row):
    idx = str(row['idx'])
    category = row['category']
    path = f"images/train/{category}/{idx}.jpg"
    return path

X_y_train['path'] = X_y_train.apply(path,axis=1)
X_y_test['path'] = X_y_test.apply(path,axis=1)

In [ ]:
import os
import shutil
import numpy as np

# Set the paths
train_base_directory = 'images/train/'  # Replace with the actual train path
test_base_directory = 'images/train/'  # Replace with the actual test path

train_output_directory = 'images/train_categorized/'  # Replace with your desired train output path
test_output_directory = 'images/test_categorized/'  # Replace with your desired test output path

# Function to create folders for each category
def create_category_folders(output_directory, unique_labels):
    for label in unique_labels:
        label_path = os.path.join(output_directory, label)
        if os.path.exists(label_path):
            shutil.rmtree(label_path)  # Delete existing folder
        os.makedirs(label_path)

# Load the numpy arrays of training and testing image paths
train_image_paths = X_y_train['path'].unique()
test_image_paths = X_y_test['path'].unique()

# Extract category labels from paths
def extract_category_label(image_path):
    category = os.path.basename(os.path.dirname(image_path))
    return category

# Create folders for each category in the train output directory
unique_train_labels = np.unique([extract_category_label(path) for path in train_image_paths])
create_category_folders(train_output_directory, unique_train_labels)

# Copy train images to categorized folders
for image_path in train_image_paths:
    category = extract_category_label(image_path)
    output_folder = os.path.join(train_output_directory, category)
    shutil.copy(image_path, output_folder)

print("Train images copied to categorized folders.")

# Create folders for each category in the test output directory
unique_test_labels = np.unique([extract_category_label(path) for path in test_image_paths])
create_category_folders(test_output_directory, unique_test_labels)

# Copy test images to categorized folders
for image_path in test_image_paths:
    category = extract_category_label(image_path)
    output_folder = os.path.join(test_output_directory, category)
    shutil.copy(image_path, output_folder)

print("Test images copied to categorized folders.")

KeyboardInterrupt: 

In [91]:
# Set the paths and image size
train_data_dir = 'images/train'
image_dir = 'images/train'
image_size = (128, 128)

# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split data into training and validation
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)


# Load and preprocess training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Use the training subset of data
)


# Load and preprocess validation data
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use the validation subset of data
)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(6, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
num_epochs = 10
model.fit(train_generator, epochs=num_epochs, validation_data=validation_generator)

# Evaluate the model on test data
test_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use the validation subset of data for testing
)

eval_result = model.evaluate(test_generator)
print("Test Loss:", eval_result[0])
print("Test Accuracy:", eval_result[1])


# Save the model
model.save('cnn_model.h5')

Found 25415 images belonging to 6 classes.
Found 6351 images belonging to 6 classes.


2023-08-22 00:27:58.710213: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
795/795 [==============================] - 346s 435ms/step - loss: 1.6302 - accuracy: 0.3304 - val_loss: 1.5755 - val_accuracy: 0.3615
Epoch 2/10
795/795 [==============================] - 343s 432ms/step - loss: 1.4959 - accuracy: 0.4086 - val_loss: 1.4592 - val_accuracy: 0.4149
Epoch 3/10
795/795 [==============================] - 343s 431ms/step - loss: 1.4186 - accuracy: 0.4513 - val_loss: 1.3599 - val_accuracy: 0.4766
Epoch 4/10
795/795 [==============================] - 344s 432ms/step - loss: 1.3705 - accuracy: 0.4739 - val_loss: 1.3683 - val_accuracy: 0.4761
Epoch 5/10
795/795 [==============================] - 344s 432ms/step - loss: 1.3322 - accuracy: 0.4896 - val_loss: 1.3576 - val_accuracy: 0.4763
Epoch 6/10
795/795 [==============================] - 345s 434ms/step - loss: 1.3121 - accuracy: 0.5020 - val_loss: 1.3084 - val_accuracy: 0.4925
Epoch 7/10
795/795 [==============================] - 348s 438ms/step - loss: 1.2829 - accuracy: 0.5136 - val_loss: 1.2864 -

In [28]:
model = load_model('cnn_model.h5')

2023-08-21 19:37:44.174649: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# FE CV
### Notice that in all the next CV we will use simple XGBClassifier and we will fine-tuning it later

In [30]:
def final_foo(df):
    x1 = extract_brand(df)
    x2 = extract_description(x1)
    x3 = extract_ingredients(x2)
    x4 = extract_household(x3)
    return x4

In [67]:
X_fe_train = final_foo(X_fe_train)
X_fe_test = final_foo(X_fe_test)

In [32]:
Brand = FunctionTransformer(extract_brand)#,kw_args={'a':5})
# Description1 = FunctionTransformer(desc_indication_columns,kw_args={'b':5})
Description2 = FunctionTransformer(extract_description)
# Ingredients1 = FunctionTransformer(ingre_indication_columns,kw_args={'c':5})
Ingredients2 = FunctionTransformer(extract_ingredients)
# Household1 = FunctionTransformer(household_indication_columns,kw_args={'d':5})
Household2 = FunctionTransformer(extract_household)
Drop = FunctionTransformer(drop_columns)

In [33]:
numerical_transformer = StandardScaler()

categorical_features = ["brand", "description", "ingredients","household_serving_fulltext",'serving_size_unit']

numeric_features = ['serving_size', 'Calcium, Ca', 'Carbohydrate, by difference','Cholesterol', 'Energy', 'Fatty acids, total saturated',
                    'Fatty acids, total trans', 'Fiber, total dietary', 'Iron, Fe','Protein', 'Sodium, Na', 'Sugars, total including NLEA',
                    'Total lipid (fat)', 'Vitamin A, IU', 'Vitamin C, total ascorbic acid']

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse=False))
    ]
)


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [70]:
pipeline = Pipeline([
    # ('brand',Brand),
    # ('description2',Description2),
    # ('ingredients2',Ingredients2),
    # ('household2',Household2),
    # ('drop',Drop),
    ("preprocessor", preprocessor),
    ('classifier', XGBClassifier(random_state=42))
])

In [71]:
scores = cross_val_score(pipeline, X_fe_train, y_fe_train.values.ravel(), cv=5, scoring='accuracy')
print("Cross-validation scores:", scores)

Cross-validation scores: [0.91471915 0.91430914 0.91181296 0.90689089 0.91017227]


### Let's fit on all X_fe_train and check the test acc

In [72]:
pipeline.fit(X_fe_train, y_fe_train.values.ravel())

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['serving_size',
                                                   'Calcium, Ca',
                                                   'Carbohydrate, by '
                                                   'difference',
                                                   'Cholesterol', 'Energy',
                                                   'Fatty acids, total '
                                                   'saturated',
                                                   'Fatty acids, total trans',
                                                   'Fiber, total dietary',
                                                   'Iron, Fe', 'Protein',
                                                   'Sodium, Na',
                                                   'Sugars, total including '
                                                   'NLEA',
                                                   'Total lipid (fat)',
                                                   'Vitamin A, IU',
                                                   'Vitamin C, t...
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               objective='multi:softprob', predictor=None, ...))])

In [73]:
test_score = pipeline.score(X_fe_test, y_fe_test.values.ravel())
print("Test score:", test_score)

report = classification_report(y_fe_test, predicted_labels)
print("Classification Report:\n", report)

Test score: 0.911745406824147
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.87      0.92       364
           1       0.90      0.86      0.88       728
           2       0.90      0.93      0.92       353
           3       0.68      0.86      0.76       362
           4       0.87      0.86      0.86       507
           5       0.92      0.87      0.89       734

    accuracy                           0.87      3048
   macro avg       0.87      0.88      0.87      3048
weighted avg       0.88      0.87      0.87      3048



### Fill null values
### We will check a few methods like mean, median and KNN

In [79]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer()),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
        ]), categorical_features)
    ])

pipeline_nan = Pipeline([
    ("preprocessor", preprocessor),
    ('classifier', XGBClassifier(random_state=42))
])

param_grid = [
    {
        'preprocessor__num__imputer': [SimpleImputer(strategy='mean'), SimpleImputer(strategy='median')] + [KNNImputer(n_neighbors=k) for k in range(1, 6)]
    }
]

grid_search = GridSearchCV(pipeline_nan, param_grid, cv=5, n_jobs=1,verbose = 3,scoring = 'accuracy')

grid_search.fit(X_fe_train, y_fe_train.values.ravel())

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV 1/5] END preprocessor__num__imputer=SimpleImputer();, score=0.916 total time=  14.8s
[CV 2/5] END preprocessor__num__imputer=SimpleImputer();, score=0.908 total time=  15.3s
[CV 3/5] END preprocessor__num__imputer=SimpleImputer();, score=0.905 total time=  14.8s
[CV 4/5] END preprocessor__num__imputer=SimpleImputer();, score=0.909 total time=  15.0s
[CV 5/5] END preprocessor__num__imputer=SimpleImputer();, score=0.907 total time=  16.2s
[CV 1/5] END preprocessor__num__imputer=SimpleImputer(strategy='median');, score=0.916 total time=  16.7s
[CV 2/5] END preprocessor__num__imputer=SimpleImputer(strategy='median');, score=0.910 total time=  16.3s
[CV 3/5] END preprocessor__num__imputer=SimpleImputer(strategy='median');, score=0.906 total time=  15.9s
[CV 4/5] END preprocessor__num__imputer=SimpleImputer(strategy='median');, score=0.908 total time=  15.8s
[CV 5/5] END preprocessor__num__imputer=SimpleImputer(strategy='median'

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['serving_size',
                                                                          'Calcium, '
                                                                          'Ca',
                                                                          'Carbohydrate, '
                                                                          'by '
                                                                          'difference',
                                                                          'Cholesterol',
                                                                          'Energy',
                                                                          'Fatty '
                                                                          'acids, '
                                                                          'total '
                                                                          'saturated',
                                                                          'Fatty '
                                                                          'acids, '
                                                                          'total '
                                                                          'trans',
                                                                          'Fiber, '
                                                                          'total '
                                                                          'dietary',
                                                                          'I...
                                                      monotone_constraints=None,
                                                      n_estimators=100,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      predictor=None,
                                                      random_state=42, ...))]),
             n_jobs=1,
             param_grid=[{'preprocessor__num__imputer': [SimpleImputer(),
                                                         SimpleImputer(strategy='median'),
                                                         KNNImputer(n_neighbors=1),
                                                         KNNImputer(n_neighbors=2),
                                                         KNNImputer(n_neighbors=3),
                                                         KNNImputer(n_neighbors=4),
                                                         KNNImputer()]}],
             scoring='accuracy', verbose=3)

In [80]:
best_imputer = grid_search.best_params_['preprocessor__num__imputer']
print("Best Imputer:", type(best_imputer).__name__)
print("Best Parameters:", best_imputer.get_params())

Best Imputer: KNNImputer
Best Parameters: {'add_indicator': False, 'copy': True, 'metric': 'nan_euclidean', 'missing_values': nan, 'n_neighbors': 2, 'weights': 'uniform'}


### We got that that KNN with 2_neigh method is the best way to fill the missing values

## SMOTE
### We will fill our null values with the KNN(2)
### And since our data is imbalanced we will try some resample method

In [32]:
X_mt_train = final_foo(X_mt_train)
X_mt_test = final_foo(X_mt_test)

In [84]:
imputer = KNNImputer(n_neighbors=2)

X_mt_train[numeric_features] = imputer.fit_transform(X_mt_train[numeric_features])
X_mt_test[numeric_features] = imputer.transform(X_mt_test[numeric_features])  # Use the same imputer on the test set

numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [85]:
y_mt_train['category'].value_counts()

5    1957
1    1941
4    1353
0     970
3     966
2     941
Name: category, dtype: int64

In [90]:
from imblearn.pipeline import Pipeline as ImbPipeline

# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), categorical_features)
    ])

# Define the pipeline using ImbPipeline from imblearn
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('resampler', SMOTE()),  # Resampler included directly in the pipeline
    ('classifier', XGBClassifier(random_state=42))
])

# Hyperparameter grid
param_grid = [
    {
        'resampler__sampling_strategy': ['auto', {5: 1957, 1: 1957, 4: 1957, 0: 1957, 3: 1957, 2: 1957}],
        'resampler__k_neighbors': [1, 3, 5, 7]
    }
]

# Grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=1, verbose=3)
grid_search.fit(X_mt_train, y_mt_train.values.ravel())

# Evaluate
y_pred = grid_search.predict(X_mt_test)
print('Best Parameters:', grid_search.best_params_)
print('Accuracy:', accuracy_score(y_mt_test, y_pred))
print(classification_report(y_mt_test, y_pred))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END resampler__k_neighbors=1, resampler__sampling_strategy=auto;, score=0.909 total time=   8.5s
[CV 2/5] END resampler__k_neighbors=1, resampler__sampling_strategy=auto;, score=0.914 total time=   9.0s
[CV 3/5] END resampler__k_neighbors=1, resampler__sampling_strategy=auto;, score=0.906 total time=   9.4s
[CV 4/5] END resampler__k_neighbors=1, resampler__sampling_strategy=auto;, score=0.907 total time=  10.4s
[CV 5/5] END resampler__k_neighbors=1, resampler__sampling_strategy=auto;, score=0.912 total time=  10.2s
[CV 1/5] END resampler__k_neighbors=1, resampler__sampling_strategy={5: 1957, 1: 1957, 4: 1957, 0: 1957, 3: 1957, 2: 1957};, score=0.909 total time=  14.1s
[CV 2/5] END resampler__k_neighbors=1, resampler__sampling_strategy={5: 1957, 1: 1957, 4: 1957, 0: 1957, 3: 1957, 2: 1957};, score=0.916 total time=  18.9s
[CV 3/5] END resampler__k_neighbors=1, resampler__sampling_strategy={5: 1957, 1: 1957, 4: 1957, 0:

## MT CV
### Now will fine-tuning XGBC and 2 other popular classifiers: RF AND GRADIENT_BOOST

In [119]:
classifiers = [
    {
        'classifier': XGBClassifier(random_state=42),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__learning_rate': [0.01, 0.1, 0.2],
            'classifier__max_depth': [3, 5, 10],
            'classifier__subsample': [0.5, 0.8, 1],
            'classifier__colsample_bytree': [0.5, 0.8, 1]
        }
    },
    {
        'classifier': RandomForestClassifier(random_state=42),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10],
            'classifier__min_samples_leaf': [1, 2, 4]
        }
    },
    {
        'classifier': HistGradientBoostingClassifier(random_state=42),
        'params': {
            'classifier__max_iter': [50, 100, 200],
            'classifier__learning_rate': [0.01, 0.1, 0.2],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_leaf': [5, 10, 20]
        }
    }
]

pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('resampler', SMOTE(k_neighbors=3, sampling_strategy='auto')),  # Resampler included directly in the pipeline
    ('classifier', None)
])

# Iterate over the classifiers and perform grid search
for clf in classifiers:
    c = clf['classifier']
    pipeline.set_params(classifier=c)
    grid_search = GridSearchCV(pipeline, clf['params'], cv=5, n_jobs=1, verbose=3,scoring = 'accuracy')
    grid_search.fit(X_mt_train, y_mt_train.values.ravel())
    
    print("Best parameters:", grid_search.best_params_)
    print("Best score:", grid_search.best_score_)

    # Evaluate on the test set
    score = grid_search.score(X_mt_test, y_mt_test.values.ravel())
    print("Test set score:", score)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV 1/5] END classifier__max_depth=None, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=50;, score=0.900 total time=   0.8s
[CV 2/5] END classifier__max_depth=None, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=50;, score=0.908 total time=   0.7s
[CV 3/5] END classifier__max_depth=None, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=50;, score=0.897 total time=   0.7s
[CV 4/5] END classifier__max_depth=None, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=50;, score=0.900 total time=   0.7s
[CV 5/5] END classifier__max_depth=None, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=50;, score=0.906 total time=   0.7s
[CV 1/5] END classifier__max_depth=None, classifier__min_samples_leaf=1, classifier__min_samples_split

# from second run rf:

In [ ]:
Best parameters: {'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 200}
Best score: 0.9082187529567604
Test set score: 0.9187992125984252

# from second run gb:

In [ ]:
Best parameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': None, 'classifier__max_iter': 200, 'classifier__min_samples_leaf': 20}
Best score: 0.9126477812470432
Test set score: 0.922244094488189

# Lets summarize what we got
### filling nan values with KNN(n_neigh = 2)
### resampling with SMOTE(k_neighbors=3, sampling_strategy='auto')
#### XGB - {'classifier__colsample_bytree': 0.5, 'classifier__learning_rate': 0.2, 'classifier__max_depth': 10, 'classifier__n_estimators': 200, 'classifier__subsample': 1}
#### RF - {'classifier__max_depth': 30, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}
#### GB - {'classifier__learning_rate': 0.2, 'classifier__max_depth': 20, 'classifier__max_iter': 100, 'classifier__min_samples_leaf': 20}

In [33]:
X_train = final_foo(X_train)
X_test = final_foo(X_test)

In [38]:
classifiers = [
    {
        'classifier': XGBClassifier(colsample_bytree=0.5,learning_rate=0.2,max_depth=10,n_estimators=200,subsample=1)
    },
    {
        'classifier': RandomForestClassifier(max_depth=None,min_samples_leaf=1,min_samples_split=2,n_estimators=200)
    },
    {
        'classifier': HistGradientBoostingClassifier(learning_rate=0.1,max_depth=None,max_iter=200,min_samples_leaf=20)
    }
]

In [39]:
numerical_transformer = StandardScaler()
imputer = KNNImputer(n_neighbors=2)
X_train[numeric_features] = imputer.fit_transform(X_train[numeric_features])
X_test[numeric_features] = imputer.transform(X_test[numeric_features])

In [40]:
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('resampler', SMOTE(k_neighbors=3, sampling_strategy='auto')),  # Resampler included directly in the pipeline
    ('classifier', None)
])


for clf in classifiers:
    c = clf['classifier']
    pipeline.set_params(classifier=c)
    print(f"fitting {c}...")
    pipeline.fit(X_train, y_train.values.ravel())

    test_score = pipeline.score(X_test, y_test.values.ravel())
    print(f"Test score for {c}:", test_score)

fitting RandomForestClassifier(n_estimators=200)...
Test score for RandomForestClassifier(n_estimators=200): 0.9212722405920327
fitting HistGradientBoostingClassifier(max_iter=200)...
Test score for HistGradientBoostingClassifier(max_iter=200): 0.9219020626672965


### Nice results!
### Remember df2? Lets try it out and we'll descide

In [43]:
X_p = df2.loc[:,df2.columns != 'category']
y_p = df2.loc[:,['category']]

y_p['category'].replace(['cakes_cupcakes_snack_cakes', 'candy', 'chips_pretzels_snacks', 'chocolate',
 'cookies_biscuits', 'popcorn_peanuts_seeds_related_snacks'],
                        [0, 1, 2, 3, 4, 5], inplace=True)

X_p_train, X_p_test, y_p_train, y_p_test = train_test_split(X_p, y_p, test_size=0.2, random_state=42, stratify=y_p)

In [44]:
X_p_train = final_foo(X_p_train)
X_p_test = final_foo(X_p_test)

In [45]:
numerical_transformer = StandardScaler()
imputer = KNNImputer(n_neighbors=2)
X_p_train[numeric_features] = imputer.fit_transform(X_p_train[numeric_features])
X_p_test[numeric_features] = imputer.transform(X_p_test[numeric_features])

In [46]:
for clf in classifiers:
    c = clf['classifier']
    pipeline.set_params(classifier=c)
    print(f"fitting {c}...")
    pipeline.fit(X_p_train, y_p_train.values.ravel())

    test_score = pipeline.score(X_p_test, y_p_test.values.ravel())
    print(f"Test score for {c}:", test_score)

fitting RandomForestClassifier(n_estimators=200)...
Test score for RandomForestClassifier(n_estimators=200): 0.9257246376811594
fitting HistGradientBoostingClassifier(max_iter=200)...
Test score for HistGradientBoostingClassifier(max_iter=200): 0.9245923913043478


In [34]:
X = final_foo(X)
data_test = final_foo(data_test)

In [35]:
X1 = X.copy()
data_test1 = data_test.copy()
y1 = y.copy()

In [36]:
numerical_transformer = StandardScaler()
imputer = KNNImputer(n_neighbors=2)
X1[numeric_features] = imputer.fit_transform(X1[numeric_features])
data_test1[numeric_features] = imputer.transform(data_test1[numeric_features])

In [37]:
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('resampler', SMOTE(k_neighbors=3, sampling_strategy='auto')),
    ('classifier', RandomForestClassifier(max_depth=None,min_samples_leaf=1,min_samples_split=2,n_estimators=200))
])

In [38]:
pipeline.fit(X1, y1.values.ravel())
preds1 = pipeline.predict(data_test1)

data_test1['pred_cat'] = pipeline.predict(data_test1)

data_test1['pred_cat'].replace([0, 1, 2, 3, 4, 5],
                              ['cakes_cupcakes_snack_cakes', 'candy', 'chips_pretzels_snacks', 'chocolate','cookies_biscuits', 'popcorn_peanuts_seeds_related_snacks'],
                              inplace=True)

data_test1.loc[:,['idx','pred_cat']]#.to_csv('model01.csv',index=False)

,idx,pred_cat
0,13,cakes_cupcakes_snack_cakes
1,15,chips_pretzels_snacks
2,16,cookies_biscuits
3,23,chips_pretzels_snacks
4,40,chips_pretzels_snacks
...,...,...
3520,35215,chips_pretzels_snacks
3521,35241,popcorn_peanuts_seeds_related_snacks
3522,35255,candy
3523,35258,chocolate


In [66]:
X = final_foo(X)

In [67]:
data_test = final_foo(data_test)

In [70]:
numerical_transformer = StandardScaler()
imputer = KNNImputer(n_neighbors=2)
X[numeric_features] = imputer.fit_transform(X[numeric_features])
data_test[numeric_features] = imputer.transform(data_test[numeric_features])

In [73]:
pipeline.fit(X, y.values.ravel())
preds = pipeline.predict(data_test)

In [78]:
data_test['pred_cat'] = pipeline.predict(data_test)

In [80]:
data_test['pred_cat'].replace([0, 1, 2, 3, 4, 5],
                              ['cakes_cupcakes_snack_cakes', 'candy', 'chips_pretzels_snacks', 'chocolate','cookies_biscuits', 'popcorn_peanuts_seeds_related_snacks'],
                              inplace=True)



In [81]:
data_test.loc[:,['idx','pred_cat']].to_csv('model01.csv',index=False)

,idx,brand,description,ingredients,serving_size,serving_size_unit,household_serving_fulltext,"Alcohol, ethyl",Biotin,"Calcium, Ca",...,cookies_biscuits_biscuit,cookies_biscuits_macaroon,cookies_biscuits_waffle,popcorn_peanuts_seeds_related_snacks_tbsp,popcorn_peanuts_seeds_related_snacks_kernel,popcorn_peanuts_seeds_related_snacks_popcorn,popcorn_peanuts_seeds_related_snacks_almond,popcorn_peanuts_seeds_related_snacks_shell,popcorn_peanuts_seeds_related_snacks_pecan,pred_cat
0,13,None,description_cakes_cupcakes_snack_cakes_brownie,ingredients_chocolate_chocolate,39.0,g,None,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,cakes_cupcakes_snack_cakes
1,15,None,description_chips_pretzels_snacks_potato,None,28.0,g,None,NaN,NaN,143.0,...,0,0,0,0,0,0,0,0,0,chips_pretzels_snacks
2,16,None,description_cookies_biscuits_cookie,ingredients_cakes_cupcakes_snack_cakes_leavening,28.0,g,household_serving_fulltext_cookies_biscuits_co...,NaN,NaN,36.0,...,0,0,0,0,0,0,0,0,0,cookies_biscuits
3,23,None,description_chips_pretzels_snacks_tortilla chips,None,28.0,g,household_serving_fulltext_chips_pretzels_snac...,NaN,NaN,71.0,...,0,0,0,0,0,0,0,0,0,chips_pretzels_snacks
4,40,None,description_chips_pretzels_snacks_potato chips,ingredients_chips_pretzels_snacks_potatoes,28.0,g,None,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,chips_pretzels_snacks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3520,35215,None,description_chips_pretzels_snacks_potato chips,ingredients_chips_pretzels_snacks_potatoes,28.0,g,None,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,chips_pretzels_snacks
3521,35241,None,description_popcorn_peanuts_seeds_related_snac...,ingredients_popcorn_peanuts_seeds_related_snac...,28.0,g,None,NaN,NaN,143.0,...,0,0,0,0,0,0,0,0,0,popcorn_peanuts_seeds_related_snacks
3522,35255,None,description_candy_candies,ingredients_candy_red 40,16.0,g,household_serving_fulltext_cakes_cupcakes_snac...,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,candy
3523,35258,None,description_chocolate_truffle,ingredients_chocolate_cocoa butter,28.0,g,None,NaN,NaN,43.0,...,0,0,0,0,0,0,0,0,0,chocolate


# DO FINAL_FOO ON ALL DATA

# CHANGE NUMBERS TO CATEGORIES !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!